In [1]:
import pandas as pd

In [2]:
df_basic_info = pd.read_csv("../data/processed/stocks-basic-info.csv")
df_history = pd.read_csv("../data/processed/stocks-history.csv")
df_fundaments = pd.read_csv("../data/processed/stocks-fundaments.csv")
df_right_prices = pd.read_csv("../data/processed/stocks-right-prices.csv")

In [3]:
def get_sectors():
    return df_basic_info[["SETOR", "SUBSETOR", "SEGMENTO"]].drop_duplicates()

In [4]:
def get_main_ticker(tickers):
    tickers = tickers.split(";")

    for ticker in tickers:
        if ticker[4] == "4":
            return ticker

    for ticker in tickers:
        if ticker[4] == "3":
            return ticker

    return tickers[0]

In [5]:
def get_companies_by_segmento(segmento):
    df_tmp = df_basic_info[df_basic_info["SEGMENTO"] == segmento].copy()
    df_tmp["MAIN_TICKER"] = df_tmp["TICKERS"].apply(get_main_ticker)
    return df_tmp[["CD_CVM", "DENOM_COMERC", "MAIN_TICKER"]]

In [6]:
def get_latest_values_by_ticker(ticker):
    df_history_tmp = df_history[df_history["TICKER"] == ticker]
    cd_cvm = df_history_tmp.iloc[0]["CD_CVM"]

    df_fundaments_tmp = df_fundaments[df_fundaments["CD_CVM"] == cd_cvm]
    last_dt_fim_exerc = df_fundaments_tmp["DT_FIM_EXERC"].max()

    df_fundaments_tmp = df_fundaments_tmp[df_fundaments_tmp["DT_FIM_EXERC"] == last_dt_fim_exerc]
    df_fundaments_tmp_2 = df_fundaments_tmp[df_fundaments_tmp["VL_CONTA_ROLLING_YEAR"] == -1]
    df_fundaments_tmp = df_fundaments_tmp[df_fundaments_tmp["VL_CONTA_ROLLING_YEAR"] != -1]
    df_fundaments_tmp = df_fundaments_tmp.pivot(index="DT_FIM_EXERC", columns="KPI", values="VL_CONTA_ROLLING_YEAR").reset_index()
    df_fundaments_tmp_2 = df_fundaments_tmp_2.pivot(index="DT_FIM_EXERC", columns="KPI", values="VL_CONTA").reset_index()

    df_fundaments_tmp = pd.concat([df_fundaments_tmp, df_fundaments_tmp_2], axis=1)
    df_fundaments_tmp = df_fundaments_tmp.drop("DT_FIM_EXERC", axis=1)

    df_history_tmp = df_history_tmp.tail(1).reset_index(drop=True)

    df_right_prices_tmp = df_right_prices[df_right_prices["TICKER"] == ticker]
    df_right_prices_tmp = df_right_prices_tmp.drop(["CD_CVM", "TICKER"], axis=1).reset_index(drop=True)
    
    df_latest_values = pd.concat([df_history_tmp, df_right_prices_tmp, df_fundaments_tmp], axis=1)

    total_stocks = df_basic_info[df_basic_info["CD_CVM"] == cd_cvm].iloc[0]["NUM_TOTAL"]

    df_latest_values["MARKET_CAP"] = df_latest_values["PRICE"] * total_stocks

    return df_latest_values

In [41]:
def get_latest_values_by_segmento(segmento):
    df_segment = pd.DataFrame()

    for ticker in get_companies_by_segmento(segmento=segmento)["MAIN_TICKER"].values:
        df_segment = pd.concat([df_segment, get_latest_values_by_ticker(ticker=ticker)])
        
    df_segment = df_segment.sort_values(by="MARKET_CAP", ascending=False)

    df_segment = df_segment.reset_index(drop=True)
    df_segment_tmp = df_segment.drop(["DATE", "CD_CVM", "TICKER", "BAZIN", "PRICE"], axis=1)

    market_cap_kpis = ["EBIT", "PROFIT", "EQUITY", "MARKET_CAP"]

    df_segment_row = df_segment_tmp.drop(market_cap_kpis, axis=1).mean().to_frame().T
    df_segment_row["TICKER"] = "SEGMENTO"

    df_segment_cols = df_segment_tmp[market_cap_kpis].sum().to_frame().T
    df_segment_cols.columns = [col + "_SEGMENTO" for col in df_segment_cols.columns]

    df_final = pd.concat([df_segment, df_segment_cols], axis=1).ffill()

    for col in df_final.columns:
        if "_SEGMENTO" in col:
            df_final[col] = df_final[col] * df_final["MARKET_CAP"] / df_final["MARKET_CAP_SEGMENTO"]
            
    df_final = df_final.drop(["DATE", "CD_CVM", "MARKET_CAP_SEGMENTO"], axis=1)
    df_final = pd.concat([df_final, df_segment_row])
    df_final = df_final.fillna(0)
    return df_final[["TICKER", "PRICE", "BAZIN", "PL", "PVP", "DIVIDEND_YIELD", "DIVIDEND_PAYOUT", 
            "ROE", "CGAR_5_YEARS", "PROFIT", "PROFIT_SEGMENTO", "EBIT", "EBIT_SEGMENTO",
            "EQUITY", "EQUITY_SEGMENTO", "MARKET_CAP"]]

In [8]:
get_sectors()

,SETOR,SUBSETOR,SEGMENTO
0,Financeiro,Intermediários Financeiros,Bancos
5,Utilidade Pública,Energia Elétrica,Energia Elétrica
7,Utilidade Pública,Água e Saneamento,Água e Saneamento


In [9]:
get_companies_by_segmento(segmento="Bancos")

,CD_CVM,DENOM_COMERC,MAIN_TICKER
0,906,BANCO BRADESCO S.A.,BBDC4
1,22616,BANCO UBS PACTUAL S/A,BPAC3
2,922,BANCO DA AMAZONIA S.A.,BAZA3
3,1023,BANCO DO BRASIL S.A.,BBAS3
4,20532,BANCO SANTANDER (BRASIL) S.A.,SANB4
14,19348,ITAÚ UNIBANCO,ITUB4


In [10]:
get_latest_values_by_ticker(ticker="BBAS3")

,DATE,CD_CVM,TICKER,PRICE,PL,DIVIDEND_YIELD,DIVIDEND_PAYOUT,PVP,BAZIN,EBIT,PROFIT,CGAR_5_YEARS,EQUITY,ROE,MARKET_CAP
0,2024-10-21,1023,BBAS3,26.559999,4.35535,0.094818,0.412965,0.829582,38.205378,4.108161e+10,3.494804e+10,0.150059,1.834792e+11,0.190474,1.522109e+11


In [42]:
get_latest_values_by_segmento(segmento="Energia Elétrica")

,TICKER,PRICE,BAZIN,PL,PVP,DIVIDEND_YIELD,DIVIDEND_PAYOUT,ROE,CGAR_5_YEARS,PROFIT,PROFIT_SEGMENTO,EBIT,EBIT_SEGMENTO,EQUITY,EQUITY_SEGMENTO,MARKET_CAP
0,ELET3,38.080002,7.568720,19.772755,0.769100,0.010604,0.209671,0.038897,-0.238640,4.443203e+09,1.006955e+10,1.113091e+10,1.901724e+10,1.142301e+11,8.034909e+10,8.785437e+10
1,EQTL3,31.900000,11.999964,12.471831,1.486651,0.014106,0.175932,0.119201,0.152833,3.192282e+09,4.563292e+09,7.897817e+09,8.618180e+09,2.678073e+10,3.641238e+10,3.981360e+10
2,CPFE3,32.720001,69.907094,6.861174,1.870994,0.084168,0.577491,0.272693,0.176248,5.494944e+09,4.321241e+09,1.065683e+10,8.161045e+09,2.015066e+10,3.448096e+10,3.770177e+10
3,EGIE3,40.799999,55.784474,7.618701,2.759209,0.062249,0.474257,0.362163,0.148381,4.369492e+09,3.815563e+09,7.659935e+09,7.206028e+09,1.206500e+10,3.044595e+10,3.328985e+10
4,CMIG4,11.030000,25.649706,5.292022,1.183218,0.165968,0.878308,0.223585,0.072663,5.964724e+09,3.617919e+09,7.510646e+09,6.832759e+09,2.667762e+10,2.886886e+10,3.156545e+10
5,NEOE3,19.230000,16.256498,5.131088,0.733404,0.043150,0.221406,0.142933,0.185403,4.549000e+09,2.675298e+09,9.876000e+09,5.052537e+09,3.182600e+10,2.134731e+10,2.334132e+10
6,TRPL4,24.510000,33.606628,5.914494,0.845822,0.089926,0.531866,0.143008,0.041619,2.730450e+09,1.850967e+09,3.743223e+09,3.495714e+09,1.909294e+10,1.476962e+10,1.614923e+10
7,TAEE4,11.550000,30.843684,7.761800,1.763664,0.099776,0.774441,0.227224,0.076908,1.537902e+09,1.368163e+09,2.492035e+09,2.583897e+09,6.768231e+09,1.091714e+10,1.193689e+10
0,SEGMENTO,0.000000,0.000000,8.852983,1.426508,0.071243,0.480421,0.191213,0.076927,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
